 # Lab: Principales plataformas de IA Generativa



 ### Objetivos del Lab

 - Gestionar credenciales de forma segura

 - Entender la diferencia entre llamadas directas a las APIs y SDKs

 - Aprender a conectar con diferentes proveedores de IA

 - Construir un chatbot multi-proveedor



 ### Proveedores que exploraremos

 - OpenAI

 - Azure OpenAI

 - Anthropic

 - Google Gemini



 ### Estructura del Lab

 1. **Preparación del entorno**

 2. **Llamada manual con requests**

 3. **SDK de OpenAI**

 4. **SDK de Azure OpenAI**

 5. **SDK de Anthropic**

 6. **SDK de Google Gemini**

 7. **Exploración de parámetros**

 8. **Problema: Chatbot financiero multi-proveedor**

 ## 1. Preparación del Entorno



 ### Instalación de dependencias



 Ejecuta la siguiente celda para instalar todas las librerías necesarias:

In [1]:
# Instalar todas las dependencias necesarias para el lab
!pip install python-dotenv requests openai anthropic google-genai


In [2]:
from dotenv import load_dotenv

 ### Estructura de archivos necesaria



 1. **Archivo `.env`** (para credenciales)

 2. **Archivo `.gitignore`** (para proteger credenciales)



 ### ⚠️ Seguridad de Credenciales



 **NUNCA** hardcodees API keys en tu código. Siempre usa variables de entorno.

 ## 2. Llamada Manual con requests



 ### Entendiendo qué ocurre "por debajo"



 Antes de usar SDKs, veamos cómo hacer una llamada directa a la API de OpenAI.



 📚 **Documentación útil:**

 - [Plataforma OpenAI](https://platform.openai.com/prompts) (Es de pago)
 - [OpenAI API Reference](https://platform.openai.com/docs/api-reference/chat/create)

 - [OpenAI Models](https://platform.openai.com/docs/models)



 ### Pasos:

 1. Obtener API key de OpenAI desde su [plataforma](https://platform.openai.com/)

 2. Configurar headers y payload

 3. Hacer la petición HTTP

 4. Procesar la respuesta JSON

In [3]:
import requests
import json
import os


# Cargar variables de entorno
load_dotenv()

def llamada_manual_openai(mensaje):
    """Llamada manual a OpenAI usando requests"""
    
    # 1. Configuración
    url = "https://api.openai.com/v1/chat/completions"
    
    # 2. Headers - Autorización
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {os.getenv('AZURE_OPENAI_API_KEY')}"
    }
    
    # 3. Payload - Datos que enviamos
    payload = {
        "model": "gpt-4o-mini",
        "messages": [
            {"role": "user", "content": mensaje}
        ],
        "temperature": 0.7,
        "max_tokens": 5
    }
    
    try:
        # 4. Hacer la petición
        response = requests.post(url, headers=headers, json=payload)
        
        # 5. Verificar que la respuesta es exitosa
        response.raise_for_status()
        
        # 6. Procesar JSON
        data = response.json()
        
        # 7. Extraer el contenido (navegación compleja del JSON)
        contenido = data['choices'][0]['message']['content']
        
        print("📤 Respuesta cruda (JSON):")
        print(json.dumps(data, indent=2))
        print("\n📨 Contenido extraído:")
        print(contenido)
        
        return contenido
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Error en la petición: {e}")
        return None
    except KeyError as e:
        print(f"❌ Error procesando respuesta: {e}")
        return None

# Prueba la función
llamada_manual_openai("Explica en una línea qué es la inteligencia artificial")


❌ Error en la petición: 401 Client Error: Unauthorized for url: https://api.openai.com/v1/chat/completions


 ## 3. SDK de OpenAI - La Forma Elegante



 ### Ventajas del SDK:

 - **Mantenimiento**: Actualizado por OpenAI

 - **Simplicidad**: Abstrae la complejidad HTTP. Menos código, más legible y tipado.


 ### **Chat Completions vs Responses API**

 OpenAI tiene dos APIs principales:

 - **Chat Completions API**: API tradicional y ampliamente usada (la que usaremos aquí)

 - **Responses API**: Nueva API más moderna con herramientas integradas (web search, file search, etc.)



 **¿Cuál usar?**

 - **Chat Completions**: Para aplicaciones tradicionales de chat y generación de texto

 - **Responses**: Para aplicaciones agénticas con herramientas integradas



 En este lab usamos **Chat Completions** porque es el estándar de la industria y OpenAI continuará soportándola indefinidamente.



 📚 **Documentación útil:**

 - [OpenAI Python SDK](https://github.com/openai/openai-python)

 - [Chat Completions API](https://platform.openai.com/docs/api-reference/chat/create)

 - [Responses vs Chat Completions](https://platform.openai.com/docs/guides/responses-vs-chat-completions)

 - [OpenAI Models](https://platform.openai.com/docs/models)

In [4]:
from openai import OpenAI
import os
from dotenv import load_dotenv
from pprint import pprint

# Cargar variables de entorno
load_dotenv()

def usar_sdk_openai(mensaje, temperatura=0.7, max_tokens=150):
    """Uso del SDK oficial de OpenAI"""
    
    # 1. Crear cliente - ¡Busca la API key automáticamente!
    client = OpenAI()
    
    try:
        # 2. Llamada simple y limpia
        response = client.chat.completions.create(
            model="gpt-4-1106-preview",
            messages=[
                {"role": "user", "content": mensaje}
            ],
            temperature=temperatura,
            max_tokens=max_tokens,
            
        )
        
        # 3. Acceso fácil al contenido
        contenido = response.choices[0].message.content
        
        print(f"🤖 OpenAI responde:")
        print(contenido)
        
        # 4. Información adicional disponible
        print(f"\n📊 Tokens usados: {response.usage.total_tokens}")
        coste = response.usage.prompt_tokens * 0.15 / 1e6  \
            + response.usage.completion_tokens * 0.6 / 1e6
        print(f"💰 Coste: ${coste:.6f}")
        
        # 5. Mostrar la respuesta completa para entender el formato
        print("\n🔍 Respuesta completa del API:")
        print("=" * 50)
        pprint(response.model_dump())
        print("=" * 50)
        
        return contenido
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

usar_sdk_openai("¿Cuál es la capital de España?", temperatura=0.1)


❌ Error: Error code: 401 - {'error': {'message': 'Incorrect API key provided: your-ope********here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}


 ## 4. SDK de Azure OpenAI





 ### Configuración en Azure:

 1. [Activar Azure for students](https://azure.microsoft.com/es-es/free/students)

 1. Ir al [catalogo de modelos de Azure AI Foundry](https://ai.azure.com/explore/models) 

 2. Desplegar un modelo

 3. Obtener endpoint y claves



 📚 **Documentación útil:**

 - [Azure OpenAI Service Documentation](https://learn.microsoft.com/azure/ai-services/openai/)

 - [Quickstart: Azure OpenAI con Python](https://learn.microsoft.com/azure/ai-services/openai/quickstart?tabs=command-line%2Cpython&pivots=programming-language-python)

 - [Diferencias entre OpenAI y Azure OpenAI](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints)

 - [Cuota de Azure for Students](https://learn.microsoft.com/es-es/azure/ai-foundry/openai/quotas-limits?tabs=REST#other-offer-types)



 ### Diferencias clave:

 - **Endpoint personalizado**: URL específica de tu recurso

 - **Nombre de despliegue**: Modelo específico que desplegaste

 - **Versión de API**: Versionado específico de Azure

In [5]:
from openai import AzureOpenAI
import os
from dotenv import load_dotenv
from pprint import pprint

# Cargar variables de entorno
load_dotenv() 

def usar_azure_openai(mensaje, temperatura=0.7):
    """Uso del SDK de Azure OpenAI"""
    
    # 1. Crear cliente de Azure - requiere configuración explícita
    client = AzureOpenAI(
        api_key=os.getenv('AZURE_OPENAI_API_KEY'),
        azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
        api_version="2024-02-01"  # Versión específica de Azure
    )
    
    try:
        # 2. Llamada idéntica a OpenAI, pero con deployment_name
        response = client.chat.completions.create(
            model=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'),  # Nombre del despliegue
            messages=[
                {"role": "user", "content": mensaje}
            ],
            temperature=temperatura,
            max_tokens=150
        )
        
        # 3. Procesamiento idéntico
        contenido = response.choices[0].message.content
        
        print(f"☁️ Azure OpenAI responde:")
        print(contenido)
        
        # 4. Mostrar la respuesta completa para entender el formato
        print("\n🔍 Respuesta completa del API:")
        print("=" * 50)
        pprint(response.model_dump())
        print("=" * 50)
        
        return contenido
        
    except Exception as e:
        print(f"❌ Error con Azure: {e}")
        return None

usar_azure_openai("Explica las ventajas de usar Azure para IA")


☁️ Azure OpenAI responde:
Azure, la plataforma de nube de Microsoft, ofrece una serie de ventajas para el desarrollo y la implementación de soluciones de inteligencia artificial (IA). A continuación, se detallan algunas de las principales ventajas de utilizar Azure para IA:

1. **Amplia gama de servicios de IA**: Azure proporciona una variedad de servicios de IA, como Azure Machine Learning, Azure Cognitive Services y Azure Bot Services, que permiten a los desarrolladores crear, entrenar e implementar modelos de IA con facilidad.

2. **Escalabilidad**: Azure permite escalar recursos de manera flexible según las necesidades del proyecto. Esto es especialmente útil para aplicaciones de IA que pueden requerir un gran poder de cómputo durante el entrenamiento de modelos o en momentos de alta demanda

🔍 Respuesta completa del API:
{'choices': [{'content_filter_results': {'hate': {'filtered': False,
                                                  'severity': 'safe'},
                      

'Azure, la plataforma de nube de Microsoft, ofrece una serie de ventajas para el desarrollo y la implementación de soluciones de inteligencia artificial (IA). A continuación, se detallan algunas de las principales ventajas de utilizar Azure para IA:\n\n1. **Amplia gama de servicios de IA**: Azure proporciona una variedad de servicios de IA, como Azure Machine Learning, Azure Cognitive Services y Azure Bot Services, que permiten a los desarrolladores crear, entrenar e implementar modelos de IA con facilidad.\n\n2. **Escalabilidad**: Azure permite escalar recursos de manera flexible según las necesidades del proyecto. Esto es especialmente útil para aplicaciones de IA que pueden requerir un gran poder de cómputo durante el entrenamiento de modelos o en momentos de alta demanda'

 ## 5. SDK de Anthropic





 ### Obtener API Key:

 1. Ir a la [consola de Anthropic](https://console.anthropic.com/)

 2. Generar API key

 3. Añadir al archivo .env



 📚 **Documentación útil:**

 - [Anthropic API Documentation](https://docs.anthropic.com/en/api/overview)

 - [Anthropic Python SDK](https://github.com/anthropics/anthropic-sdk-python)

 - [Prompt Engineering para Claude](https://docs.anthropic.com/en/docs/build-with-claude/prompt-engineering/overview)

 - [Modelos disponibles](https://docs.anthropic.com/en/docs/about-claude/models/overview)

 - [Precios de los modelos](https://docs.anthropic.com/en/docs/about-claude/pricing)

In [ ]:
import anthropic
import os
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

def usar_anthropic(mensaje, temperatura=0.7):
    """Uso del SDK de Anthropic"""
    
    # 1. Crear cliente de Anthropic
    client = anthropic.Anthropic() # Usa automáticamente la API key ANTHROPIC_API_KEY del entorno
    
    try:
        # 2. Llamada específica de Anthropic
        response = client.messages.create(
            model="claude-3-5-haiku-latest",
            max_tokens=150,
            temperature=temperatura,
            messages=[
                {"role": "user", "content": mensaje}
            ]
        )
        
        # 3. Extraer contenido
        contenido = response.content[0].text
        
        print(f"🧠 Haiku responde:")
        print(contenido)
        
        # 4. Información sobre uso
        print(f"\n📊 Tokens de entrada: {response.usage.input_tokens}")
        print(f"📊 Tokens de salida: {response.usage.output_tokens}")
        coste = response.usage.input_tokens * 0.8 / 1e6  \
            + response.usage.output_tokens * 4.0 / 1e6
        print(f"💰 Coste: ${coste:.6f}")
        
        # 5. Mostrar la respuesta completa para entender el formato
        print("\n🔍 Respuesta completa del API:")
        print("=" * 50)
        from pprint import pprint
        pprint(response.model_dump())
        print("=" * 50)
        
        return contenido
        
    except Exception as e:
        print(f"❌ Error con Anthropic: {e}")
        return None


# Prueba
usar_anthropic("¿Cuáles son las consideraciones éticas más importantes en IA?")


❌ Error con Anthropic: Error code: 401 - {'type': 'error', 'error': {'type': 'authentication_error', 'message': 'invalid x-api-key'}, 'request_id': 'req_011CUzkhpjBLPyRhEXsc8aXs'}


 ## 6. SDK de Google Gemini





 ### Obtener API Key:

 1. Ir a [Google AI Studio](https://aistudio.google.com)

 2. Crear cuenta y generar API key

 3. Añadir al archivo .env



 📚 **Documentación útil:**

 - [Google Gemini API Docs](https://ai.google.dev/gemini-api/docs)

 - [SDK google-genai](https://github.com/googleapis/python-genai)

 - [Modelos disponibles y límites](https://ai.google.dev/models/gemini)



In [7]:
from google import genai
from google.genai import types
import os
from dotenv import load_dotenv
from pprint import pprint

# Cargar variables de entorno
load_dotenv()

def usar_gemini(mensaje, temperatura=0.7):
    """Uso del SDK de Google Gemini"""
    
    try:
        # 1. Crear cliente
        client = genai.Client() # Usa automáticamente la API key GEMINI_API_KEY 
        
        # 2. Crear contenido en formato correcto
        content = types.Content(
            role='user',
            parts=[types.Part.from_text(text=mensaje)]
        )
        
        # 3. Generar contenido
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            #model="gemma-3-27b-it",
            contents=[content],
            config=types.GenerateContentConfig(
                temperature=temperatura,
        
            )
        )
        
        # 3. Extraer texto
        contenido = response.text
        
        print(f"💎 Gemini responde:")
        print(contenido)
        
        # 4. Información adicional
        if hasattr(response, 'usage_metadata'):
            print(f"\n📊 Tokens entrada: {response.usage_metadata.prompt_token_count}")
            print(f"📊 Tokens salida: {response.usage_metadata.candidates_token_count}")
        
        # 5. Mostrar la respuesta completa para entender el formato
        print("\n🔍 Respuesta completa del API:")
        print("=" * 50)
        pprint(response.model_dump())
        print("=" * 50)
        
        return contenido
        
    except Exception as e:
        print(f"❌ Error con Gemini: {e}")
        return None

# Prueba
usar_gemini("Explica muy resumidamente las ventajas de los modelos multimodales")

❌ Error con Gemini: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'API key not valid. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key not valid. Please pass a valid API key.'}]}}


 ## 7. Explorando Parámetros de los LLMs



 ### 🎛️ Parámetros más importantes:

 - **temperature**: Controla la aleatoriedad general (0.0 = determinista, 2.0 = muy creativo)

 - **max_tokens**: Límite de tokens en la respuesta

 - **top_p**: Nucleus sampling (alternativa a temperature). Controla que tokens considera el modelo (0.0-1.0)

 - **presence_penalty**: Penaliza/bonifica tokens ya mencionados (-2.0-2.0)
 
 - **frequency_penalty**: Penaliza/bonifica repetición de tokens y se acumula (-2.0-2.0)

 - **stop**: Secuencias de caracteres que pararan la generacion

 - **logprobs**: Muestra las probabilidades que el modelo asigna a cada token

 - **top_logprobs**: Número de alternativas más probables a mostrar (1-20)

 
 ### 🧪 Experimentemos con diferentes configuraciones

In [8]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import math

load_dotenv()

def demostrar_logprobs():
    """Demuestra logprobs y top_logprobs"""
    
    client = OpenAI()
    prompt = "¿Como suele ser el clima en madrid?"
    
    print(f"Prompt: '{prompt}'\n")
    print("🎯 EXPLORANDO LOG PROBABILITIES - Viendo las probabilidades de cada token")
    print("=" * 70)
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=15,
            logprobs=True,        # Activar log probabilities
            top_logprobs=3       # Mostrar top 3 tokens más probables
        )
        
        # Extraer información de logprobs
        logprobs_data = response.choices[0].logprobs
        
        if logprobs_data and logprobs_data.content:
            print(f"📤 Respuesta generada: '{response.choices[0].message.content}'\n")
            
            print("📊 ANÁLISIS TOKEN POR TOKEN:")
            print("-" * 70)
            
            for i, token_data in enumerate(logprobs_data.content):
                token = token_data.token
                logprob = token_data.logprob
                probability = round(math.e ** logprob * 100, 2)  # Convertir log a %
                
                print(f"Token {i+1}: '{token}'")
                print(f"   📊 Probabilidad: {probability}% (logprob: {logprob:.5f})")
                
                if token_data.top_logprobs:
                    print(f"   🏆 Top 3 alternativas más probables:")
                    for j, alt in enumerate(token_data.top_logprobs):
                        alt_prob = round(math.e ** alt.logprob * 100, 3)
                        print(f"      {j+1}. '{alt.token}' → {alt_prob}%")
                
                print()
        
        # Mostrar respuesta completa para debugging
        print("\n🔍 Respuesta completa del API (logprobs):")
        print("=" * 50)
            
    except Exception as e:
        print(f"❌ Error: {e}")


def comparar_temperature_vs_top_p():
    """Compara temperature vs top_p"""
    
    client = OpenAI()
    prompt = "¿Cómo suele ser el clima en Madrid?"
    
    print(f"Prompt: '{prompt}'\n")
    
    # Usando solo temperature
    print("🌡️ USANDO TEMPERATURE:")
    for temp in [0.0, 1, 2]:
        print(f"   Temperature {temp}:")
        for i in range(10):
            try:
                response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[{"role": "user", "content": prompt}],
                    temperature=temp,
                    max_tokens=20
                )
                resultado = response.choices[0].message.content.strip()
                print(f"     • '{resultado}'")
            except Exception as e:
                print(f"     • Error: {e}")
        print()
    
    # Usando solo top_p
    print("🎯 USANDO TOP-P (temperature=1.0 fijo):")
    for top_p in [0, 0.5, 1.0]:
        print(f"   Top-p {top_p}:")
        for i in range(10):
            try:
                response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[{"role": "user", "content": prompt}],
                    temperature=1.0,
                    top_p=top_p,
                    max_tokens=20
                )
                resultado = response.choices[0].message.content.strip()
                print(f"     • '{resultado}'")
            except Exception as e:
                print(f"     • Error: {e}")
        print()

def demostrar_frequency_penalty():
    """Demuestra frequency_penalty"""
    
    client = OpenAI()
    prompt = "Escribe sobre la salud. Usa la palabra 'salud' muchas veces."
    
    print(f"Prompt: '{prompt}'\n")
    
    for penalty in [-2, 0, 2]:
        print(f"🔄 Frequency penalty: {penalty}")
        
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7,
                frequency_penalty=penalty,
                max_tokens=500
            )
            
            resultado = response.choices[0].message.content
            repeticiones = resultado.lower().count('salud')
            
            print(f"   📤 {resultado[:80]}...")
            print(f"   🔢 'salud' aparece {repeticiones} veces")
            
        except Exception as e:
            print(f"   ❌ Error: {e}")
        
        print()

def demostrar_stop_tokens():
    """Demuestra el parámetro stop"""
    
    client = OpenAI()
    prompt = "Cuenta una historia corta"
    
    print(f"Prompt: '{prompt}'\n")
    print("🛑 USANDO STOP TOKENS - Parando en el primer punto")
    print("=" * 60)
    
    try:
        # Sin stop token
        print("📝 SIN stop token:")
        response_sin_stop = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=100
        )
        
        resultado_sin_stop = response_sin_stop.choices[0].message.content
        print(f"   📤 {resultado_sin_stop}")
        print(f"   🔚 Finish reason: {response_sin_stop.choices[0].finish_reason}")
        print()
        
        # Con stop token: punto
        print("📝 CON stop token '.':")
        response_con_stop = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=100,
            stop=["."]  # Para en el primer punto
        )
        
        resultado_con_stop = response_con_stop.choices[0].message.content
        print(f"   📤 {resultado_con_stop}")
        print(f"   🔚 Finish reason: {response_con_stop.choices[0].finish_reason}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

# Ejecutar demostraciones
print("=" * 50)
demostrar_logprobs()
comparar_temperature_vs_top_p()
print("=" * 50)
demostrar_frequency_penalty()
print("=" * 50)
demostrar_stop_tokens()


Prompt: '¿Como suele ser el clima en madrid?'

🎯 EXPLORANDO LOG PROBABILITIES - Viendo las probabilidades de cada token
❌ Error: Error code: 401 - {'error': {'message': 'Incorrect API key provided: your-ope********here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Prompt: '¿Cómo suele ser el clima en Madrid?'

🌡️ USANDO TEMPERATURE:
   Temperature 0.0:
     • Error: Error code: 401 - {'error': {'message': 'Incorrect API key provided: your-ope********here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
     • Error: Error code: 401 - {'error': {'message': 'Incorrect API key provided: your-ope********here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
     • Error: Error c

 ## 8. Problema: Chatbot Multi-Proveedor con Streaming



 ### 🎯 Objetivo

 Crear un **Asistente Financiero** que puede usar Azure OpenAI o Google Gemini con streaming en tiempo real.



 ### 📋 Requisitos

 1. **Selección de proveedor**: El usuario elige entre 'azure' o 'gemini'

 2. **Chat interactivo**: Mantiene conversación hasta que el usuario escriba '/salir'

 3. **Streaming**: Respuestas en tiempo real

 4. **Contexto**: Mantiene el historial de la conversación

 5. **Personalidad**: Actúa como asesor financiero profesional



 📚 **Documentación útil para el proyecto:**

 - [Google Gemini API Docs](https://ai.google.dev/gemini-api/docs)

 - [SDK google-genai](https://github.com/googleapis/python-genai)

 - [OpenAI API Reference](https://platform.openai.com/docs/api-reference/chat/create)



 ### 💡 Consejo:

 - Al imprimir respuestas en streaming, usa `print(..., end="", flush=True)` para que aparezcan en tiempo real.



 ## 🎉 ¡Felicitaciones!



 Has completado el lab de **Acceso a modelos privados de IA Generativa**. Ahora tienes las habilidades para:



 ✅ Conectar con múltiples proveedores de IA

 ✅ Gestionar credenciales de forma segura

 ✅ Construir aplicaciones conversacionales

